<a href="https://colab.research.google.com/github/manikcb/sem_project/blob/main/cyberbully3004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install -q -U "tensorflow-text==2.8.*"

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd


In [10]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Untitled folder/mydata3.csv")
df.head(10)

,message,category
0,â??@JuanPee24: â??@_MrHandsomeee: â??@tayyoung...,harass
1,@politicalmoxie no dumb ass @Conservativeind c...,harass
2,"@AndrewS82805912 Andrew, Andrew, Andrew.. Tsk ...",harass
3,"Some dumb fuck really had the nerve to type ""f...",harass
4,All I dumb fuck black got all these different ...,harass
5,Bruh they even said he didnâ??t get it threw y...,harass
6,i'm a dumb nigger that loves to leave my twitt...,harass
7,You know there are people out there who like @...,not_harass
8,"@kebesays no, my blog post. :)",not_harass
9,"Kat and Andre celebrating bad cooking, bad att...",not_harass


In [5]:
df_harass = df[df['category']=='harass']
df_not_harass = df[df['category']=='not_harass']

In [6]:
df_not_harass_downsampled = df_not_harass.sample(df_harass.shape[0],replace=True)
df_balanced = pd.concat([df_not_harass_downsampled, df_harass])
df_balanced['category'].value_counts()


not_harass    710
harass        710
Name: category, dtype: int64

In [7]:
df_balanced['harass']=df_balanced['category'].apply(lambda x: 1 if x=='harass' else 0)
df_balanced.sample(2)

,message,category,harass
1015,Preciso fazer um script de 10 minutos sobre bu...,not_harass,0
979,Nothing more disgusting than niggers who enabl...,harass,1


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['message'],df_balanced['harass'], stratify=df_balanced['harass'])
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [9]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)
model.fit(X_train, y_train, epochs=12)
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

Epoch 1/12
34/34 [==============================] - 664s 19s/step - loss: 0.7127 - accuracy: 0.5258 - precision: 0.5246 - recall: 0.5414
Epoch 2/12
34/34 [==============================] - 646s 19s/step - loss: 0.6203 - accuracy: 0.6629 - precision: 0.6454 - recall: 0.7218
Epoch 3/12
34/34 [==============================] - 629s 18s/step - loss: 0.5591 - accuracy: 0.7643 - precision: 0.7646 - recall: 0.7632
Epoch 4/12
34/34 [==============================] - 623s 18s/step - loss: 0.5186 - accuracy: 0.7953 - precision: 0.8019 - recall: 0.7838
Epoch 5/12
34/34 [==============================] - 623s 18s/step - loss: 0.4910 - accuracy: 0.8235 - precision: 0.8039 - recall: 0.8553
Epoch 6/12
34/34 [==============================] - 625s 18s/step - loss: 0.4560 - accuracy: 0.8432 - precision: 0.8288 - recall: 0.8647
Epoch 7/12
34/34 [==============================] - 624s 18s/step - loss: 0.4265 - accuracy: 0.8732 - precision: 0.8656 - recall: 0.8835
Epoch 8/12
34/34 [=======================

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,